In [1]:
import h2o


In [2]:

# Explore a typical Data Science workflow with H2O and Python
#
# Goal: assist the manager of CitiBike of NYC to load-balance the bicycles
# across the CitiBike network of stations, by predicting the number of bike
# trips taken from the station every day.  Use 10 million rows of historical
# data, and eventually add weather data.


# Connect to a cluster
h2o.init()




--------------------------  ---------------------------
H2O cluster uptime:         25 seconds 458 milliseconds
H2O cluster version:        0.3.0.99999
H2O cluster name:           cliffc
H2O cluster total nodes:    1
H2O cluster total memory:   7.67 GB
H2O cluster total cores:    16
H2O cluster allowed cores:  16
H2O cluster healthy:        True
--------------------------  ---------------------------



In [3]:
# Pick either the big or the small demo.
# Big data is 10M rows
small_test = ["bigdata/laptop/citibike-nyc/2013-10.csv"]
big_test =   ["bigdata/laptop/citibike-nyc/2013-07.csv",
              "bigdata/laptop/citibike-nyc/2013-08.csv",
              "bigdata/laptop/citibike-nyc/2013-09.csv",
              "bigdata/laptop/citibike-nyc/2013-10.csv",
              "bigdata/laptop/citibike-nyc/2013-11.csv",
              "bigdata/laptop/citibike-nyc/2013-12.csv",
              "bigdata/laptop/citibike-nyc/2014-01.csv",
              "bigdata/laptop/citibike-nyc/2014-02.csv",
              "bigdata/laptop/citibike-nyc/2014-03.csv",
              "bigdata/laptop/citibike-nyc/2014-04.csv",
              "bigdata/laptop/citibike-nyc/2014-05.csv",
              "bigdata/laptop/citibike-nyc/2014-06.csv",
              "bigdata/laptop/citibike-nyc/2014-07.csv",
              "bigdata/laptop/citibike-nyc/2014-08.csv"]

# ----------

# 1- Load data - 1 row per bicycle trip.  Has columns showing the start and end
# station, trip duration and trip start time and day.  The larger dataset
# totals about 10 million rows
print "Import and Parse bike data"
data = h2o.import_frame(path=big_test)




Import and Parse bike data

Parse Progress: [##################################################] 100%
Veckeys [{u'URL': None, u'type': u'Key<Vec>', u'name': u'$04ff01000000ffffffff$nfs:\\C:\\Users\\cliffc\\Desktop\\h2o-dev\\bigdata\\laptop\\citibike-nyc\\2013-07.csv', u'__meta': {u'schema_name': u'VecKeyV1', u'schema_version': 1, u'schema_type': u'Key<Vec>'}}, {u'URL': None, u'type': u'Key<Vec>', u'name': u'$04ff02000000ffffffff$nfs:\\C:\\Users\\cliffc\\Desktop\\h2o-dev\\bigdata\\laptop\\citibike-nyc\\2013-07.csv', u'__meta': {u'schema_name': u'VecKeyV1', u'schema_version': 1, u'schema_type': u'Key<Vec>'}}, {u'URL': None, u'type': u'Key<Vec>', u'name': u'$04ff03000000ffffffff$nfs:\\C:\\Users\\cliffc\\Desktop\\h2o-dev\\bigdata\\laptop\\citibike-nyc\\2013-07.csv', u'__meta': {u'schema_name': u'VecKeyV1', u'schema_version': 1, u'schema_type': u'Key<Vec>'}}, {u'URL': None, u'type': u'Key<Vec>', u'name': u'$04ff04000000ffffffff$nfs:\\C:\\Users\\cliffc\\Desktop\\h2o-dev\\bigdata\\laptop\\cit

In [4]:
# ----------

# 2- light data munging: group the bike starts per-day, converting the 10M rows
# of trips to about 140,000 station&day combos - predicting the number of trip
# starts per-station-per-day.

# Convert start time to: Day since the Epoch
startime = data["starttime"]
secsPerDay=1000*60*60*24
data["Days"] = (startime/secsPerDay).floor()
data.describe()



Rows: 10407546 Cols: 16
               tripduration    starttime          stoptime           start station id    start station name    start station latitude    start station longitude    end station id    end station name    end station latitude    end station longitude    bikeid         usertype        birth year     gender          Days
-------------  --------------  -----------------  -----------------  ------------------  --------------------  ------------------------  -------------------------  ----------------  ------------------  ----------------------  -----------------------  -------------  --------------  -------------  --------------  -------------
type           int             time               time               int                 enum                  real                      real                       int               enum                real                    real                     int            enum            int            int             int
mins          

In [5]:
# Now do a monster Group-By.  Count bike starts per-station per-day.  Ends up
# with about 340 stations times 400 days (140,000 rows).  This is what we want
# to predict.
ddplycols=["Days","start station name"]
bpd = h2o.ddply(data[ddplycols],ddplycols,"(%nrow)")  # Compute bikes-per-day
bpd["C1"]._name = "bikes" # Rename column from generic name



In [6]:
# Quantiles: the data is fairly unbalanced; some station/day combos are wildly
# more popular than others.
print "Quantiles of bikes-per-day"
bpd["bikes"].quantile().show()



Quantiles of bikes-per-day
  Row ID    bikes (first 9 row(s))
--------  ------------------------
       1                         2
       2                        11
       3                        26
       4                        35
       5                        58
       6                        89
       7                       107
       8                       157
       9                       291



In [7]:
# A little feature engineering
# Add in month-of-year (seasonality; fewer bike rides in winter than summer)
secs = bpd["Days"]*secsPerDay
bpd["Month"]     = secs.month()
# Add in day-of-week (work-week; more bike rides on Sunday than Monday)
bpd["DayOfWeek"] = secs.dayOfWeek()
print "Bikes-Per-Day"
bpd.describe()


Bikes-Per-Day
Rows: 139261 Cols: 5
               Days           start station name    bikes          Month          DayOfWeek
-------------  -------------  --------------------  -------------  -------------  -------------
type           int            enum                  int            enum           enum
mins           15887.0        0.0                   1.0            0.0            0.0
mean           16099.9758008  169.640760873         74.7341035897  5.68177738204  3.0059384896
maxs           16314.0        339.0                 680.0          11.0           6.0
sigma          123.635133897  98.50295732           64.1243887565  3.20373100216  2.00302100015
zero_count     0              428                   0              9949           19880
missing_count  0              0                     0              0              0


Internal FluidVec compression/distribution summary:

    Chunk Type    Count    Count Percentage    Size      Size Percentage
--  ------------  -------  

In [8]:
# ----------
# 3- Fit a model on train; using test as validation

# Function for doing class test/train/holdout split
def split_fit_predict(data):
  # Classic Test/Train split
  r = data['Days'].runif()   # Random UNIForm numbers, one per row
  train = data[  r  < 0.6]
  test  = data[(0.6 <= r) & (r < 0.9)]
  hold  = data[ 0.9 <= r ]
  print "Training data has",train.ncol(),"columns and",train.nrow(),"rows, test has",test.nrow(),"rows, holdout has",hold.nrow()
  
  # Run GBM
  gbm = h2o.gbm(x           =train.drop("bikes"),
                y           =train     ["bikes"],
                validation_x=test .drop("bikes"),
                validation_y=test      ["bikes"],
                ntrees=500, # 500 works well
                max_depth=6,
                learn_rate=0.1)

  # Run DRF
  drf = h2o.random_forest(x =train.drop("bikes"),
                y           =train     ["bikes"],
                validation_x=test .drop("bikes"),
                validation_y=test      ["bikes"],
                ntrees=500, # 500 works well
                max_depth=50)

  # Run GLM
  glm = h2o.glm(x           =train.drop("bikes"),
                y           =train     ["bikes"],
                validation_x=test .drop("bikes"),
                validation_y=test      ["bikes"],
                dropNA20Cols=True)
  #glm.show()
  
  
  # ----------
  # 4- Score on holdout set & report
  train_r2_gbm = gbm.model_performance(train).r2()
  test_r2_gbm  = gbm.model_performance(test ).r2()
  hold_r2_gbm  = gbm.model_performance(hold ).r2()
  print "GBM R2 TRAIN=",train_r2_gbm,", R2 TEST=",test_r2_gbm,", R2 HOLDOUT=",hold_r2_gbm
  
  train_r2_drf = drf.model_performance(train).r2()
  test_r2_drf  = drf.model_performance(test ).r2()
  hold_r2_drf  = drf.model_performance(hold ).r2()
  print "DRF R2 TRAIN=",train_r2_drf,", R2 TEST=",test_r2_drf,", R2 HOLDOUT=",hold_r2_drf
  
  train_r2_glm = glm.model_performance(train).r2()
  test_r2_glm  = glm.model_performance(test ).r2()
  hold_r2_glm  = glm.model_performance(hold ).r2()
  print "GLM R2 TRAIN=",train_r2_glm,", R2 TEST=",test_r2_glm,", R2 HOLDOUT=",hold_r2_glm
  # --------------



In [9]:
# Split the data (into test & train), fit some models and predict on the holdout data
split_fit_predict(bpd)
# Here we see an r^2 of 0.91 for GBM, and 0.71 for GLM.  This means given just
# the station, the month, and the day-of-week we can predict 90% of the
# variance of the bike-trip-starts.


Training data has 5 columns and 83795 rows, test has 41643 rows, holdout has 13823

gbm Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

glm Model Build Progress: [##################################################] 100%
GBM R2 TRAIN= 0.96601042827 , R2 TEST= 0.922711534678 , R2 HOLDOUT= 0.918842405638
DRF R2 TRAIN= 0.801796515639 , R2 TEST= 0.7919404403 , R2 HOLDOUT= 0.788198629144
GLM R2 TRAIN= 0.711006232492 , R2 TEST= 0.706796249967 , R2 HOLDOUT= 0.700993160367


In [10]:
# ----------
# 5- Now lets add some weather
# Load weather data
wthr1 = h2o.import_frame(path=["bigdata/laptop/citibike-nyc/31081_New_York_City__Hourly_2013.csv",
                               "bigdata/laptop/citibike-nyc/31081_New_York_City__Hourly_2014.csv"])
# Peek at the data
wthr1.describe()




Parse Progress: [##################################################] 100%
Veckeys [{u'URL': None, u'type': u'Key<Vec>', u'name': u'$04ff01000000ffffffff$nfs:\\C:\\Users\\cliffc\\Desktop\\h2o-dev\\bigdata\\laptop\\citibike-nyc\\31081_New_York_City__Hourly_2013.csv', u'__meta': {u'schema_name': u'VecKeyV1', u'schema_version': 1, u'schema_type': u'Key<Vec>'}}, {u'URL': None, u'type': u'Key<Vec>', u'name': u'$04ff02000000ffffffff$nfs:\\C:\\Users\\cliffc\\Desktop\\h2o-dev\\bigdata\\laptop\\citibike-nyc\\31081_New_York_City__Hourly_2013.csv', u'__meta': {u'schema_name': u'VecKeyV1', u'schema_version': 1, u'schema_type': u'Key<Vec>'}}, {u'URL': None, u'type': u'Key<Vec>', u'name': u'$04ff03000000ffffffff$nfs:\\C:\\Users\\cliffc\\Desktop\\h2o-dev\\bigdata\\laptop\\citibike-nyc\\31081_New_York_City__Hourly_2013.csv', u'__meta': {u'schema_name': u'VecKeyV1', u'schema_version': 1, u'schema_type': u'Key<Vec>'}}, {u'URL': None, u'type': u'Key<Vec>', u'name': u'$04ff04000000ffffffff$nfs:\\C:\\Users

In [11]:
# Lots of columns in there!  Lets plan on converting to time-since-epoch to do
# a 'join' with the bike data, plus gather weather info that might affect
# cyclists - rain, snow, temperature.  Alas, drop the "snow" column since it's
# all NA's.  Also add in dew point and humidity just in case.  Slice out just
# the columns of interest and drop the rest.
wthr2 = wthr1["Year Local","Month Local","Day Local","Hour Local","Dew Point (C)","Humidity Fraction","Precipitation One Hour (mm)","Temperature (C)","Weather Code 1/ Description"]
wthr2["Precipitation One Hour (mm)"]._name = "Rain (mm)" # Shorter column name
wthr2["Weather Code 1/ Description"]._name = "WC1" # Shorter column name
wthr2.describe()
# Much better!  



Rows: 17520 Cols: 9
               Year Local      Month Local    Day Local      Hour Local     Dew Point (C)    Humidity Fraction    Rain (mm)      Temperature (C)    WC1
-------------  --------------  -------------  -------------  -------------  ---------------  -------------------  -------------  -----------------  -------------
type           int             int            int            int            real             real                 real           real               enum
mins           2013.0          1.0            1.0            0.0            -26.7            0.1251               0.0            -15.6              0.0
mean           2013.5          6.52602739726  15.7205479452  11.5           4.31304646766    0.596736389159       1.37993010753  12.5789090701      6.16417322835
maxs           2014.0          12.0           31.0           23.0           24.4             1.0                  26.924         36.1               11.0
sigma          0.500014270017  3.44794972385  

In [12]:
# Filter down to the weather at Noon
wthr3 = wthr2[ wthr2["Hour Local"]==12 ]



In [13]:
# Lets now get Days since the epoch... we'll convert year/month/day into Epoch
# time, and then back to Epoch days.  Need zero-based month and days, but have
# 1-based.
wthr3["msec"] = h2o.H2OVec.mktime(year=wthr3["Year Local"], month=wthr3["Month Local"]-1, day=wthr3["Day Local"]-1, hour=wthr3["Hour Local"])
secsPerDay=1000*60*60*24
wthr3["Days"] = (wthr3["msec"]/secsPerDay).floor()
wthr3.describe()
# msec looks sane (numbers like 1.3e12 are in the correct range for msec since
# 1970).  Epoch Days matches closely with the epoch day numbers from the
# CitiBike dataset.  


Rows: 730 Cols: 11
               Year Local      Month Local    Day Local      Hour Local    Dew Point (C)    Humidity Fraction    Rain (mm)      Temperature (C)    WC1            msec              Days
-------------  --------------  -------------  -------------  ------------  ---------------  -------------------  -------------  -----------------  -------------  ----------------  -------------
type           int             int            int            int           real             real                 real           real               enum           int               int
mins           2013.0          1.0            1.0            12.0          -26.7            0.1723               0.0            -13.9              0.0            1.3570704e+12     15706.0
mean           2013.5          6.52602739726  15.7205479452  12.0          4.23012379642    0.539728198074       1.53125714286  14.0687757909      5.18181818182  1.3885608526e+12  16070.5
maxs           2014.0          12.0       

In [14]:

# Lets drop off the extra time columns to make a easy-to-handle dataset.
wthr4 = wthr3.drop("Year Local").drop("Month Local").drop("Day Local").drop("Hour Local").drop("msec")


In [15]:
# Also, most rain numbers are missing - lets assume those are zero rain days
rain = wthr4["Rain (mm)"]
rain[rain == None ] = 0


In [16]:
# ----------
# 6 - Join the weather data-per-day to the bike-starts-per-day
print "Merge Daily Weather with Bikes-Per-Day"
bpd_with_weather = bpd.merge(wthr4,allLeft=True,allRite=False)
bpd_with_weather.describe()
bpd_with_weather.show()



Merge Daily Weather with Bikes-Per-Day
Rows: 139261 Cols: 10
               Days           start station name    bikes          Month          DayOfWeek      Humidity Fraction    Rain (mm)        Temperature (C)    WC1            Dew Point (C)
-------------  -------------  --------------------  -------------  -------------  -------------  -------------------  ---------------  -----------------  -------------  ---------------
type           int            enum                  int            enum           enum           real                 real             real               enum           real
mins           15887.0        0.0                   1.0            0.0            0.0            0.1723               0.0              -13.9              0.0            -26.7
mean           16099.9758008  169.640760873         74.7341035897  5.68177738204  3.0059384896   0.532494425803       0.0860139306769  15.6334205959      5.09840544434  5.47825137402
maxs           16314.0        339.0    

In [17]:
# 7 - Test/Train split again, model build again, this time with weather
split_fit_predict(bpd_with_weather)


Training data has 10 columns and 83325 rows, test has 41930 rows, holdout has 14006

gbm Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

glm Model Build Progress: [##################################################] 100%
GBM R2 TRAIN= 0.965496639937 , R2 TEST= 0.927997436911 , R2 HOLDOUT= 0.929164174219
DRF R2 TRAIN= 0.854580917016 , R2 TEST= 0.844209490168 , R2 HOLDOUT= 0.848629904163
GLM R2 TRAIN= 0.723581688957 , R2 TEST= 0.720152442756 , R2 HOLDOUT= 0.730959520174
